In [1]:
# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import pickle
import gzip
import json

In [2]:
with open('../input/2020-digix-advertisement-ctr-prediction/test_data_B.csv', 'rb') as inputfile:
    test_B = pd.read_csv(inputfile, sep='|',index_col=0)

In [3]:
with open('../input/ctr-6-train-test-split-0/list_mapping_dict.json', 'rb') as inputfile2:
    list_mapping_dict = json.load(inputfile2)

In [4]:
with open('../input/ctr-6-split-sgd-batch-class-weight/sgd_model.pkl', 'rb') as inputfile3:
    sgd_model = pickle.load(inputfile3)

In [5]:
# fix mapping dict (convert keys to integer)
list_mapping_dict_fixed = []
for i, mapping_dict in enumerate(list_mapping_dict):
    if i==32:
        list_mapping_dict_fixed.append(mapping_dict)
        continue
    mapping_dict_fixed = {int(k):v for k,v in mapping_dict.items()}
    list_mapping_dict_fixed.append(mapping_dict_fixed)

In [6]:
testvalues = np.sort(test_B["task_id"].unique()).tolist()

In [7]:
map_values=list(list_mapping_dict_fixed[2].keys())

In [8]:
map_minus_test = [i for i in map_values if i not in testvalues]
len(map_minus_test)

2737

In [9]:
test_minus_map = [i for i in testvalues if i not in map_values]
len(test_minus_map)

574

In [10]:
for i, col in enumerate(test_B.columns.tolist()):
    encoded=test_B[col].map(list_mapping_dict_fixed[i+1])
    downcasted = pd.to_numeric(encoded , downcast='float')
    test_B[col+'_tenc']=downcasted

In [11]:
test_B.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 1 to 1000000
Data columns (total 70 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   uid                               1000000 non-null  int64  
 1   task_id                           1000000 non-null  int64  
 2   adv_id                            1000000 non-null  int64  
 3   creat_type_cd                     1000000 non-null  int64  
 4   adv_prim_id                       1000000 non-null  int64  
 5   dev_id                            1000000 non-null  int64  
 6   inter_type_cd                     1000000 non-null  int64  
 7   slot_id                           1000000 non-null  int64  
 8   spread_app_id                     1000000 non-null  int64  
 9   tags                              1000000 non-null  int64  
 10  app_first_class                   1000000 non-null  int64  
 11  app_second_class                  100

In [12]:
#drop original cols
test_B = test_B.drop(columns=[test_B.columns[i] for i in range(0,36)])
#drop pt_d_tenc         
test_B = test_B.drop(columns=["pt_d_tenc"])

In [13]:
test_B.isna().sum()

task_id_tenc                        245232
adv_id_tenc                         245232
creat_type_cd_tenc                       0
adv_prim_id_tenc                     77607
dev_id_tenc                          60581
inter_type_cd_tenc                       0
slot_id_tenc                             0
spread_app_id_tenc                   71408
tags_tenc                            22924
app_first_class_tenc                     0
app_second_class_tenc                    0
age_tenc                                 0
city_tenc                                0
city_rank_tenc                           0
device_name_tenc                         4
device_size_tenc                         5
career_tenc                              0
gender_tenc                              0
net_type_tenc                            0
residence_tenc                           0
his_app_size_tenc                        0
his_on_shelf_time_tenc                   0
app_score_tenc                           0
emui_dev_te

In [14]:
#fill na
for col_name in test_B.columns: 
    test_B[col_name].fillna(test_B[col_name].mean(), inplace=True)

In [15]:
test_B.isna().sum()

task_id_tenc                        0
adv_id_tenc                         0
creat_type_cd_tenc                  0
adv_prim_id_tenc                    0
dev_id_tenc                         0
inter_type_cd_tenc                  0
slot_id_tenc                        0
spread_app_id_tenc                  0
tags_tenc                           0
app_first_class_tenc                0
app_second_class_tenc               0
age_tenc                            0
city_tenc                           0
city_rank_tenc                      0
device_name_tenc                    0
device_size_tenc                    0
career_tenc                         0
gender_tenc                         0
net_type_tenc                       0
residence_tenc                      0
his_app_size_tenc                   0
his_on_shelf_time_tenc              0
app_score_tenc                      0
emui_dev_tenc                       0
list_time_tenc                      0
device_price_tenc                   0
up_life_dura

In [16]:
# predictions
y_pred = sgd_model.predict_proba(test_B)
submission_arr = y_pred[:,1].round(decimals=6)
id_arr = np.arange(1,len(submission_arr))

In [17]:
# export submission
submissiondf=pd.DataFrame(zip(id_arr,submission_arr),columns=["id","probability"])
submissiondf.to_csv("submission.csv",index=False)